# MRS GABA voxel location, overlap, segmentation

#Enter ID and Activity Group

In [5]:
subcode=S43_PL

In [6]:
Activity_group=Inactive

In [7]:
case $Activity_group in 
    Active) 
        echo "Participant from active group"
        cd /Volumes/Lacie/Ex_rTMS_study/Data/Active/ 
        datadir=/Volumes/LaCie/Ex_rTMS_study/Data/Active/ 
        echo $datadir
        ;;
    Inactive)
        echo "Participant from Inactive group"
        cd /Volumes/LaCie/Ex_rTMS_study/Data/Inactive/
        datadir=/Volumes/LaCie/Ex_rTMS_study/Data/Inactive/
        echo $datadir
        ;;
esac
subdir=$datadir$subcode

Participant from Inactive group
/Volumes/LaCie/Ex_rTMS_study/Data/Inactive/


In [9]:
for i in {pre,post};
do 
/Applications/MRIcron/dcm2nii -n -g $subdir/MRI/dicom_series/$i/T1/*-1-*.dcm
cp $subdir/MRI/dicom_series/$i/T1/2*mpragesagp2iso1ADNI*.nii.gz $subdir/MRI/t1_$i.nii.gz
echo finished_$i
done
#gzip $subdir/MRI/t1_$i.nii

Chris Rorden's dcm2nii :: 30APR2016 64bit BSD License 
reading preferences file /Users/joshua_hendrikse/.dcm2nii/dcm2nii.ini
Data will be exported to /Volumes/LaCie/Ex_rTMS_study/Data/Inactive/S43_PL/MRI/dicom_series/pre/T1
Validating 195 potential DICOM images.
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95446]= Offset95446 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95446]= Offset95446 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95448]= Offset95448 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95450]= Offset95450 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95450]= Offset95450 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95450]= Offset95450 Length0
FFFE,E000,Image Fragment [0@992]= Offset992 Length0
FFFE,E000,Image Fragment [0@95450]= O

In [10]:
fslreorient2std $subdir/MRI/t1_pre.nii.gz $subdir/MRI/t1_pre_orient
fslreorient2std $subdir/MRI/t1_post.nii.gz $subdir/MRI/t1_post_orient

In [11]:
fsl_anat -i $subdir/MRI/t1_pre_orient.nii.gz

Fri Jun 29 21:53:46 AEST 2018
Reorienting to standard orientation
Fri Jun 29 21:53:56 AEST 2018
Automatically cropping the image
Starting Single Image Segmentation
T1-weighted image
Imagesize : 192 x 240 x 170
Pixelsize : 1 x 1 x 1

1 4.7185
2 4.99721
3 5.27811
KMeans Iteration 0
KMeans Iteration 1
KMeans Iteration 2
KMeans Iteration 3
KMeans Iteration 4
KMeans Iteration 5
KMeans Iteration 6
KMeans Iteration 7
KMeans Iteration 8
KMeans Iteration 9
KMeans Iteration 10
KMeans Iteration 11
KMeans Iteration 12
KMeans Iteration 13
KMeans Iteration 14
Tanaka Iteration 0 bias field 10
Tanaka-inner-loop-iteration=0 MRFWeightsTotal=1.80681e+07 beta=0.02
Tanaka-inner-loop-iteration=1 MRFWeightsTotal=1.82926e+07 beta=0.02
Tanaka-inner-loop-iteration=2 MRFWeightsTotal=1.83008e+07 beta=0.02
Tanaka-inner-loop-iteration=3 MRFWeightsTotal=1.83012e+07 beta=0.02
Tanaka-inner-loop-iteration=4 MRFWeightsTotal=1.83012e+07 beta=0.02
 CLASS 1 MEAN 82.2784 STDDEV 112.356 CLASS 2 MEAN 141.211 STDDEV 33.1814 CL

In [12]:
fsleyes $subdir/MRI/t1_pre_orient.anat/T1_to_MNI_nonlin.nii.gz &

[1] 10454


In [13]:
cp $subdir/MRI/t1_pre_orient.anat/T1_biascorr.nii.gz $subdir/MRI/t1_pre_biascorr.nii.gz
bet $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/t1_pre_biascorr_brain -m -c 85 105 99 -f 0.4
fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/t1_pre_biascorr_brain -l "Green" -t 0.5 &

[1]+  Done                    fsleyes $subdir/MRI/t1_pre_orient.anat/T1_to_MNI_nonlin.nii.gz
[1] 10567


In [14]:
flirt -dof 6 -in $subdir/MRI/t1_pre_orient -ref $subdir/MRI/t1_pre_biascorr -omat $subdir/MRI/t1_pre_orient_to_biascorr.mat -out $subdir/MRI/t1_pre_orient_coreg_biascorr
flirt -dof 6 -in $subdir/MRI/t1_post_orient -ref $subdir/MRI/t1_pre_biascorr -omat $subdir/MRI/t1_post_orient_to_biascorr.mat -out $subdir/MRI/t1_post_orient_coreg_biascorr

[1]+  Done                    fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/t1_pre_biascorr_brain -l "Green" -t 0.5


In [15]:
for i in {pre,post};
do
for j in {HP,PTL,SMA};
do
voxel_mask_from_rda.sh $subdir/MRI/t1_${i}_orient.nii.gz $subdir/MRI/MRS/${i}/${j}1.rda $subdir/MRI/${i}_${j}1
flirt -in $subdir/MRI/${i}_${j}1 -ref $subdir/MRI/t1_pre_biascorr -applyxfm -init $subdir/MRI/t1_${i}_orient_to_biascorr.mat -out $subdir/MRI/${i}_${j}1_reg_biascorr
done
done


POS= 26.935526 29.685413 -15.492984
VOX= 15.000000 20.000000 30.000000
ROW= 0.006339 0.510237 -0.860010
COL= 0.999954 0.002962 0.009128
1.000000 1.000000 1.000000
ZED= .0072047 -.8600282 .5101948
POS= 30.793423 67.841440 35.736234
VOX= 19.000000 25.000000 19.000000
ROW= -0.564879 0.462129 -0.683629
COL= 0.446203 -0.525833 -0.724156
1.000000 1.000000 1.000000
ZED= -.6941280 -.7140978 -.0908287
POS= 3.609233 3.752013 48.303360
VOX= 19.000000 25.000000 19.000000
ROW= 0.030156 0.958115 0.284793
COL= 0.997612 -0.011138 -0.068163
1.000000 1.000000 1.000000
ZED= -.0621359 .2861684 .9561628
POS= 25.567880 21.402103 2.610520
VOX= 15.000000 20.000000 30.000000
ROW= -0.090208 -0.485640 0.869492
COL= -0.994233 0.094755 -0.050226
1.000000 1.000000 1.000000
ZED= -.0579970 -.8690083 .4913869
POS= 29.301338 64.183855 49.802804
VOX= 19.000000 25.000000 19.000000
ROW= -0.503924 0.483828 -0.715521
COL= 0.450698 -0.559389 -0.695669
1.000000 1.000000 1.000000
ZED= -.7368386 -.6730481 -.0638292
POS= -1.8590

In [16]:
fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_HP1_reg_biascorr -l "Yellow" $subdir/MRI/post_HP1_reg_biascorr -l "Green" -t 0.6 &
fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_PTL1_reg_biascorr -l "Yellow" $subdir/MRI/post_PTL1_reg_biascorr -l "Green" -t 0.6 &
fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_SMA1_reg_biascorr -l "Yellow" $subdir/MRI/post_SMA1_reg_biascorr -l "Green" -t 0.6 &

[1] 11607
[2] 11610
[3] 11613


In [17]:
for i in {pre,post};
do
for j in {HP,PTL,SMA};
do
echo ${i}_${j}
get_voxel_contents.sh $subdir/MRI/t1_pre_biascorr_brain $subdir/MRI/${i}_${j}1_reg_biascorr $subdir/MRI/${i}_${j}1_seg
done
done

[1]   Done                    fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_HP1_reg_biascorr -l "Yellow" $subdir/MRI/post_HP1_reg_biascorr -l "Green" -t 0.6
[2]-  Done                    fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_PTL1_reg_biascorr -l "Yellow" $subdir/MRI/post_PTL1_reg_biascorr -l "Green" -t 0.6
[3]+  Done                    fslview_deprecated $subdir/MRI/t1_pre_biascorr.nii.gz $subdir/MRI/pre_SMA1_reg_biascorr -l "Yellow" $subdir/MRI/post_SMA1_reg_biascorr -l "Green" -t 0.6
pre_HP
partial volume segmentation in progress
grey_percent = 0.528885
total_vol = 8303.000000
volume of grey = 4391.332155
white_percent = 0.435057
csf_percent = 0.036058
pre_PTL
partial volume segmentation in progress
grey_percent = 0.380111
total_vol = 8390.000000
volume of grey = 3189.131290
white_percent = 0.519085
csf_percent = 0.100805
pre_SMA
partial volume segmentation in progress
grey_percent = 0.518128
total_vol = 8367.000000
volume of gr

In [18]:
for j in {HP,PTL,SMA};
do
echo ${j}
fslmaths $subdir/MRI/pre_${j}1_reg_biascorr -add $subdir/MRI/post_${j}1_reg_biascorr -uthr 2 -thr 1 -bin $subdir/MRI/overlap_${j}1
fslstats $subdir/MRI/overlap_${j}1 -V
fslstats $subdir/MRI/post_${j}1_reg_biascorr -V
done

HP
8303 8303.000000 
10030 10030.000000 
PTL
8405 8405.000000 
10459 10459.000000 
SMA
8927 8927.000000 
9954 9954.000000 
